In [100]:
import glob
import pickle
import numpy
import os
from music21 import converter
from music21 import instrument
from music21 import note
from music21 import chord
from music21 import stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from midi2audio import FluidSynth

In [76]:
# printing the current working directory
print(os.getcwd())

c:\Users\Danil\Documents\GitHub\LSTM-music-gen\model


# **Training The Model**

### The midi files in the directory are dissected into notes and chords and then agregated into a single file in string fromat 

In [60]:
notes = []

for file in glob.glob("c:/MIDIfile/directory/*.mid"): # Specify MIDI file directory
    midi = converter.parse(file)
    print("Parsing %s" % file)

    notes_to_parse = None
    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

with open('../notes', 'wb') as filepath:
    pickle.dump(notes, filepath)

Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\80sPop(Medley).mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\ActionRadar.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\Aerodynamic.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\AllAround.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\AllFallsDown.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\ANeverendingDream.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\AnotherRace.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\ArmyOfMe.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\AroundTheWorld(2).mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\AroundTheWorld.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\Aurora.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\AxelF.mid
Parsing c:/Users/Danil/Documents/GitHub/LSTM-music-gen/data\BabysGotATemper.mid
Parsi

In [106]:
# load the notes for consistency
with open('../notes', 'rb') as filepath:
    notes = pickle.load(filepath)

### Transform the notes sequences into the adequate input format for the LSTM model 

In [108]:
# get amount of pitch names
n_vocab = len(set(notes))
   
# sequence length should be changed after experimenting with different numbers
sequence_length = 300

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

### Create the LSTM architechture

In [109]:
model = Sequential() # initialize the sequntial model
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True)) # 512 units layer, preprocessd input data, all sequences are in the outputs for stacking each layer
model.add(Dropout(0.3)) # tu prevent  overfitting 30% of the inputs are set to zero
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256)) # connect every neuron among all layers and change the vector’s dimensions. 
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax')) # convert outputs to a probability range from 0 to 1
model.compile(loss='categorical_crossentropy', optimizer='rmsprop') # categorical_crossentropy is efficient for multi-class classification problems, such as the labelling of many notes 
# rmsprop is a prominent optimizer 

In [71]:
# Model Summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 512)           1052672   
                                                                 
 dropout_3 (Dropout)         (None, 30, 512)           0         
                                                                 
 lstm_4 (LSTM)               (None, 30, 512)           2099200   
                                                                 
 dropout_4 (Dropout)         (None, 30, 512)           0         
                                                                 
 lstm_5 (LSTM)               (None, 512)               2099200   
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_5 (Dropout)         (None, 256)              

### Train the Model

In [110]:
# experiment with different epoch sizes and batch sizes
model.fit(network_input, network_output, epochs=1, batch_size=256)

403/403 [==============================] - 25309s 63s/step - loss: 4.6071


# **Generating the final midi_file**


###  Extrapolate notes through the Neural Network based on a random sequence of inputed notes

In [113]:
# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []

# generate 1000 notes
for note_index in range(1000):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction) # numpy array of predictions
    result = int_to_note[index] # indexing the note with the highest probability
    prediction_output.append(result) # that note is the prediction output

### Converting the generated prediction into the new note sequence

In [114]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.SnareDrum()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.SnareDrum()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5



### Create the midi_file using the new note sequence 

In [115]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='~/LSTM-music-gen/generated_output/generate_midi_1ep_256batch.mid') # Specify the output directory 

'C:/Users/Danil/Documents/GitHub/LSTM-music-gen/generated_output/generate_midi_1ep_256batch.mid'